In [1]:
>>> import certifi
>>> import urllib3
>>> http = urllib3.PoolManager(
cert_reqs='CERT_REQUIRED',
ca_certs=certifi.where())

In [2]:
from lxml import html  
import json
import requests
import json,re
from dateutil import parser as dateparser
from time import sleep
import csv

In [3]:
def ParseReviews(asin):
	for i in range(5):
		try:
			#This script has only been tested with Amazon.com
			amazon_url  = 'http://www.amazon.com/dp/'+asin
			# Add some recent user agent to prevent amazon from blocking the request 
			# Find some chrome user agent strings  here https://udger.com/resources/ua-list/browser-detail?browser=Chrome
			headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
			page = requests.get(amazon_url,headers = headers,verify=True)
			page_response = page.text

			parser = html.fromstring(page_response)
			XPATH_AGGREGATE = '//span[@id="acrCustomerReviewText"]'
			XPATH_REVIEW_SECTION_1 = '//div[contains(@id,"reviews-summary")]'
			XPATH_REVIEW_SECTION_2 = '//div[@data-hook="review"]'

			XPATH_AGGREGATE_RATING = '//table[@id="histogramTable"]//tr'
			XPATH_PRODUCT_NAME = '//h1//span[@id="productTitle"]//text()'
			XPATH_PRODUCT_PRICE  = '//span[@id="priceblock_ourprice"]/text()'
			
			raw_product_price = parser.xpath(XPATH_PRODUCT_PRICE)
			product_price = ''.join(raw_product_price).replace(',','')

			raw_product_name = parser.xpath(XPATH_PRODUCT_NAME)
			product_name = ''.join(raw_product_name).strip()
			total_ratings  = parser.xpath(XPATH_AGGREGATE_RATING)
			reviews = parser.xpath(XPATH_REVIEW_SECTION_1)
			if not reviews:
				reviews = parser.xpath(XPATH_REVIEW_SECTION_2)
			ratings_dict = {}
			reviews_list = []
			
			if not reviews:
				raise ValueError('unable to find reviews in page')

			#grabing the rating  section in product page
			for ratings in total_ratings:
				extracted_rating = ratings.xpath('./td//a//text()')
				if extracted_rating:
					rating_key = extracted_rating[0] 
					raw_raing_value = extracted_rating[1]
					rating_value = raw_raing_value
					if rating_key:
						ratings_dict.update({rating_key:rating_value})
			#Parsing individual reviews
			for review in reviews:
				XPATH_RATING  = './/i[@data-hook="review-star-rating"]//text()'
				XPATH_REVIEW_HEADER = './/a[@data-hook="review-title"]//text()'
				XPATH_REVIEW_POSTED_DATE = './/span[@data-hook="review-date"]//text()'
				XPATH_REVIEW_TEXT_1 = './/div[@data-hook="review-collapsed"]//text()'
				XPATH_REVIEW_TEXT_2 = './/div//span[@data-action="columnbalancing-showfullreview"]/@data-columnbalancing-showfullreview'
				XPATH_REVIEW_COMMENTS = './/span[@data-hook="review-comment"]//text()'
				XPATH_AUTHOR  = './/a[contains(@href,"/profile/")]/parent::span//text()'
				XPATH_REVIEW_TEXT_3  = './/div[contains(@id,"dpReviews")]/div/text()'
				raw_review_author = review.xpath(XPATH_AUTHOR)
				raw_review_rating = review.xpath(XPATH_RATING)
				raw_review_header = review.xpath(XPATH_REVIEW_HEADER)
				raw_review_posted_date = review.xpath(XPATH_REVIEW_POSTED_DATE)
				raw_review_text1 = review.xpath(XPATH_REVIEW_TEXT_1)
				raw_review_text2 = review.xpath(XPATH_REVIEW_TEXT_2)
				raw_review_text3 = review.xpath(XPATH_REVIEW_TEXT_3)

				author = ' '.join(' '.join(raw_review_author).split()).strip('By')

				#cleaning data
				review_rating = ''.join(raw_review_rating).replace('out of 5 stars','')
				review_header = ' '.join(' '.join(raw_review_header).split())
				review_posted_date = dateparser.parse(''.join(raw_review_posted_date)).strftime('%d %b %Y')
				review_text = ' '.join(' '.join(raw_review_text1).split())

				#grabbing hidden comments if present
				if raw_review_text2:
					json_loaded_review_data = json.loads(raw_review_text2[0])
					json_loaded_review_data_text = json_loaded_review_data['rest']
					cleaned_json_loaded_review_data_text = re.sub('<.*?>','',json_loaded_review_data_text)
					full_review_text = review_text+cleaned_json_loaded_review_data_text
				else:
					full_review_text = review_text
				if not raw_review_text1:
					full_review_text = ' '.join(' '.join(raw_review_text3).split())

				raw_review_comments = review.xpath(XPATH_REVIEW_COMMENTS)
				review_comments = ''.join(raw_review_comments)
				review_comments = re.sub('[A-Za-z]','',review_comments).strip()
				review_dict = {
									#'review_comment_count':review_comments,
									'product':product_name,
									'price':product_price,
									'review_text':full_review_text,
									'review_posted_date':review_posted_date,
									'review_header':review_header,
									'review_rating':review_rating
									#'review_author':author

								}
				reviews_list.append(review_dict)

			data = {
						#'ratings':ratings_dict,
						'reviews':reviews_list
						#'url':amazon_url,
						#'price':product_price,
						#'name':product_name
					}
			return data
		except ValueError:
			print ("Retrying to get the correct response")
	
	return {"error":"failed to process the page","asin":asin}

In [4]:
##JSON dump works up to here##
data = ParseReviews('B00BUW9OZO')

Retrying to get the correct response
Retrying to get the correct response


In [5]:
datafile = 'dataAmazon.csv'

In [6]:
data = dict(data)
data.keys()

dict_keys(['reviews'])

In [7]:
list(data.items())

[('reviews',
  [{'product': 'Angel Soft Bath Tissue, 18 Double Rolls',
    'price': '$14.99',
    'review_text': "This has to be the worst toilet paper I have ever used. It's not even as good as Grandma's outhouse catalog pages. No absorbency what-so-ever. You need about eight sheets for the absorbency of three of a good paper. Spend a little more and get much more for your money. Extra large cardboard core is also very wasteful. ~~~Janice ~~~",
    'review_posted_date': '18 Jun 2016',
    'review_header': 'Lousy bathroom product',
    'review_rating': '1.0 '},
   {'product': 'Angel Soft Bath Tissue, 18 Double Rolls',
    'price': '$14.99',
    'review_text': "These are the same packs I would buy at walmart but about a dollar cheaper. And with my Prime membership, having it delivered right to my door in 2 days makes it well worth the buy. It's my favorite brand in that it's right in the middle of that sweet spot of not too soft, not too strong. It's not like the discount tree-bark stuf

In [8]:
import pandas as pd

In [9]:
pd.DataFrame.from_dict(list(data.items()))

,0,1
0,reviews,"[{'product': 'Angel Soft Bath Tissue, 18 Doubl..."


In [15]:
with open(datafile, 'w') as f:
    fieldnames = ['Product', 'Price', 'Review_Text', 'Review_Date', 'Title', 'Rating']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    data2 = [dict(zip(fieldnames, [a, b,])) for a, b in data.items()]
    writer.writerows(data2)

In [14]:
with open(datafile, 'w', newline='', encoding='utf8') as f:
    w = csv.writer(f, delimiter='\t')
    w.writerow(data.items())

In [18]:
data2['reviews']

TypeError: list indices must be integers or slices, not str